# 多输入通道
> 每个通道都有一个卷积核，结果是所有通道卷积结果的和，输出是一个单通道
<img alt="image.png" height="500" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-24_16.18.50.png" width="550"/>

# 多输出通道
> 每个输出通道可以识别特定模式
> 输入通道核识别并组合输入中的模式
<img alt="image.png" height="450" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-24_16.19.30.png" width="750"/>

# 1 * 1 卷积层
1 * 1的卷积层是一个受欢迎的选择，它不识别空间模式，只是融合通道
<img alt="image.png" height="450" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-24_16.28.33.png" width="750"/>

# 多输入多输出通道

# 多输入通道互相关运算

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K) : 
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))      # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起

In [25]:
# 验证
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

K.shape, corr2d_multi_in(X, K)

(torch.Size([2, 2, 2]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

# 计算多个通道的输出的互相关函数

In [26]:
def corr2d_multi_in_out(X, K) : 
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)   # k是一个3D的tensor，并在维度0上stack起来

# K之前是一个3维的tensor，现在做多输出通道，那么K要变成4维的tensor, 比如在这里做成3输出通道
K = torch.stack((K, K +  1, K + 2), 0)  # 通过将核张量K与K+1(K中每个元素加1)和K+2连接起来，构造了一个具有3个输出通道的卷积核。
K.shape, corr2d_multi_in_out(X, K)

(torch.Size([3, 2, 2, 2]),
 tensor([[[ 56.,  72.],
          [104., 120.]],
 
         [[ 76., 100.],
          [148., 172.]],
 
         [[ 96., 128.],
          [192., 224.]]]))

# 1 x 1卷积

In [29]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [31]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
float(torch.abs(Y1 - Y2).sum()) < 1e-6

True